In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
sns.set(font_scale=1)
import warnings
warnings.filterwarnings('ignore')

In [4]:
def to_tokenize(text_Series, dict_change_word): # input은 '기사내용'을 pd.Series 타입으로 줌. out도 Series 타입.
    import re

    # 1. URL 부분 제거.
    print("\nURL 제거 과정.\n")
    text_Series = [re.sub('((http(s?))\:\/\/)([0-9a-zA-Z\-]+\.*\/*(\\n)*)+|url|URL', '', text) for text in tqdm(text_Series)]

    # 2. e-mail 주소 제거.
    print("\ne-mail 제거 과정.\n")
    text_Series = [re.sub('(\<.[a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4})', '', text) for text in tqdm(text_Series)] 

    # 3. 고유어 변경
    print("\n고유어 변경 과정.\n")
    #text_Series = text_Series.apply(lambda x: change_hiphen_word(x, dict_change_word))
    text_Series = [change_hiphen_word(text, dict_change_word) for text in tqdm(text_Series)]
    
    # 4. 특수문자 제거. 모두 제거.
    print("\n특수문자 제거 과정.\n")
    text_Series = [re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', text) for text in tqdm(text_Series)]

    # 5. 문자 외 모두 제거(공백, \, 숫자 사라짐)
    print("\n문자 외 제거 과정.\n")
    text_Series = [re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', text) for text in tqdm(text_Series)]

    return text_Series


In [5]:
#dict_change_word에 해당하는 단어를 변경하는 함수

def change_hiphen_word(text, dictionary):
    com_list = [".replace('{}', '{}')".format(word, dictionary[word]) for word in dictionary.keys() if word in text]
    com_list.insert(0, "'{}'.format(text)")
    command = ''.join(com_list)
    text = eval(command)

    return text

In [6]:
#하나로 합쳐진 데이터 불러오기

news_entire = pd.read_csv('train_data.csv',encoding = 'cp949')

news_entire = news_entire.drop_duplicates('contents')

news_entire

,Unnamed: 0,date,title,contents
0,24,2020.07.19.,"[코로나19X사회공헌 下] 물품 기부 VS 캠페인?행사…유통업계, 사회공헌의 양...",리조이스 캠페인은 여성 우울증 예방과 인식 개선에 기여하는 기업의 사회적 책임(CS...
1,62,2020.11.26.,[2020 CSR] “디자인 재능 기부 통해 골목상권 활기 꿈꿨죠”,어워드 &amp; CSR 필름페스티벌'에서 ‘우리 가게 전담 디자이너’ 사업이 시작...
2,86,2020.04.10.,[뉴스락 특별기획] 코로나19 여파 속 '노블레스 오블리주'…10대 기업 기부 행...,코로나19 발발 직후 전국재해구호협회에 10억원을 기부한 LG그룹은 지주사 LG를 ...
3,110,2020.10.16.,"'세계식량의 날' 맞은 식품업계, 기부하고 절약 캠페인 펼치고",기업사회공헌(CSR) 전문 스토어다. 아이배냇은 앞서 지난 4월에도 코로나19 극복...
4,134,2020.04.10.,[뉴스락 특별기획] 코로나19 여파 속 '노블레스 오블리주'…10대 기업 기부 행...,코로나19 발발 직후 전국재해구호협회에 10억원을 기부한 LG그룹은 지주사 LG를 ...
...,...,...,...,...
37933,188658,2020.12.05.,서산시 부석면 희망 2021 나눔 캠페인 행사 개최,"20포대, 현대건설에서 쌀10kg 60포대를 어려운 이웃을 위해 사용해달라고 기부했..."
37934,188786,2020.10.23.,(주)망고스틴그룹 ‘디지털 캠페인 전문 회사’로 도약,"머스트잇, 동아오츠카, 신라면세점, 현대건설, 토스 등 다양한 브랜드의 디지털 캠페..."
37936,192696,2020.01.16.,"현대제철, 선물세트·상품권 등 전달…재래시장 장보기 캠페인","이어 박종성 부사장은 ""현대제철은 앞으로도 지역민을 위한 지속적인 나눔 실천을 통해..."
37937,192704,2020.01.16.,"현대제철, 설 선물 400가구에 전달…재래시장 활성화 캠페인도","현대제철 임직원 봉사단은 5만 원 상당의 선물세트 400개와 전통시장 상품권, 당진..."


In [7]:
#딕셔너리 통합 (고유어들을 하나의 주제로 분류할 수 있게끔 설정)

dict_change_word= {'경북':'지역사회','광주':'지역사회','대구' : '지역사회' 
                   ,'외국인':'취약계층', '장애인':'취약계층', 
                   '사회적 책임':'사회공헌','CSR':'사회공헌', '고객정보':'개인정보',
                   'ESG':'지속가능', '이노베이션':'혁신',
                   '동반성장' : '상생', '갑질' : '불공정', '기부금' : '기부',
                   '산재':'산업재해', '산업 재해' : '산업재해'}

In [8]:
#토큰화하기 위해 데이터클리닝을 해줌

news_entire_to_tokenize = pd.DataFrame(to_tokenize(news_entire['title'], dict_change_word))

news_entire_to_tokenize.reset_index(drop=True, inplace=True)

news_entire_to_tokenize.head(10)

 33%|███████████████████████▌                                                | 12378/37918 [00:00<00:00, 123042.04it/s]


URL 제거 과정.


e-mail 제거 과정.


고유어 변경 과정.



100%|████████████████████████████████████████████████████████████████████████| 37918/37918 [00:00<00:00, 339882.71it/s]


특수문자 제거 과정.


문자 외 제거 과정.



,0
0,코로나 사회공헌 물품 기부 캠페인 행사 유통업계 사회공헌의 양
1,사회공헌 디자인 재능 기부 통해 골목상권 활기 꿈꿨죠
2,뉴스락 특별기획 코로나 여파 속 노블레스 오블리주 대 기업 기부 행
3,세계식량의 날 맞은 식품업계 기부하고 절약 캠페인 펼치고
4,뉴스락 특별기획 코로나 여파 속 노블레스 오블리주 대 기업 기부 행
5,기업과 재단 편 기부를 남 몰래 하라 공익사업 잘하고도 깜깜이 지
6,기업과 재단 편 기부를 남 몰래 하라 공익사업 잘하고도 깜깜이 지
7,대형 건설사 올해 기부 억 원 전년
8,대기업 총수의 이웃사랑 통큰 기부 봇물
9,


In [9]:
#Stopword에 추가할 리스트 (조사, 명사, 일부 동사)

stopword =  pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt")['아'].tolist()

stopwords_post = ['이', '가', '께서', '에서', '서', '이다', '의', '을', '를', '이', '가', '에', '에게', '께', '한테', '에서', '에게서', '로서', '로써', '고', '이라고', '라고', '와', '과', '이랑', '에', '같이', '처럼', '만큼', '만치', '야',
                  '아', '여', '이여', '이시여', '와', '과', '하고', '이다', '이며', '에다', '에다가', '이랑', '랑']

my_stopwords = pd.read_table("stopword_list.txt")['목록'].values.tolist()

stopword = set(stopword + stopwords_post + my_stopwords)

In [10]:
#Tagging 작업

from konlpy.tag import Okt

okt = Okt()

news_tagged = news_entire_to_tokenize[0].apply(lambda x : okt.morphs(x, stem = True)).tolist()

#불용어 제거

news_tagged = [[word for word in word_list if word not in stopword] for word_list in tqdm(news_tagged)]

#한글자 제거

news_tagged = [[word for word in word_list if len(word) > 1] for word_list in tqdm(news_tagged)]

100%|████████████████████████████████████████████████████████████████████████| 37918/37918 [00:00<00:00, 837332.68it/s]


In [11]:
#아래와 같이 태깅 작업이 완료됨

news_tagged[0]

['사회', '공헌', '물품', '기부', '캠페인', '행사', '유통업', '사회', '공헌']

In [12]:
#태깅 완료 후 제목을 태깅한 것으로 대체해줌

news_ready_for_LDA = news_entire
news_ready_for_LDA['title'] = news_tagged

news_ready_for_LDA.head(10)

,Unnamed: 0,date,title,contents
0,24,2020.07.19.,"[사회, 공헌, 물품, 기부, 캠페인, 행사, 유통업, 사회, 공헌]",리조이스 캠페인은 여성 우울증 예방과 인식 개선에 기여하는 기업의 사회적 책임(CS...
1,62,2020.11.26.,"[사회, 공헌, 디자인, 재능, 기부, 통해, 골목, 상권, 활기, 꿈꾸다]",어워드 &amp; CSR 필름페스티벌'에서 ‘우리 가게 전담 디자이너’ 사업이 시작...
2,86,2020.04.10.,"[특별, 기획, 여파, 노블레스, 오블리주, 기업, 기부]",코로나19 발발 직후 전국재해구호협회에 10억원을 기부한 LG그룹은 지주사 LG를 ...
3,110,2020.10.16.,"[식량, 맞다, 식품, 업계, 기부, 절약, 캠페인, 펼치다]",기업사회공헌(CSR) 전문 스토어다. 아이배냇은 앞서 지난 4월에도 코로나19 극복...
4,134,2020.04.10.,"[특별, 기획, 여파, 노블레스, 오블리주, 기업, 기부]",코로나19 발발 직후 전국재해구호협회에 10억원을 기부한 LG그룹은 지주사 LG를 ...
5,139,2020.08.21.,"[기업, 재단, 기부, 몰래, 하라, 공익, 사업, 깜깜, 지다]",특히 김남호 DB그룹 회장은 취임사에서 기업의 사회적 역할을 강조하고 나서 향후 D...
6,144,2020.08.21.,"[기업, 재단, 기부, 몰래, 하라, 공익, 사업, 깜깜, 지다]",지난해에도 DB손해보험이 10억원을 출연했으나 재단측이 홈페이지에 공개한 '2019...
7,202,2020.11.17.,"[대형, 건설, 기부]",GS건설은 기부금 항목 미공시로 제외 ⓒ 시사오늘 올해 대형 건설사들이 전년보다 기...
8,398,2020.12.15.,"[대기업, 총수, 이웃, 사랑, 통큰, 기부, 봇물]",삼성그룹 관계자는 “NGO 사업을 금전적으로 후원하는데 그치지 않고 기업의 사회적 ...
9,404,2020.12.09.,[],"LG는 8일 서울 중구 사랑의 열매 회관에서 예종석 사회복지공동모금회장, 이방수 ㈜..."


In [13]:
#전체 데이터프레임 ['기사내용']에 전처리 완료된 데이터 저장
news_ready_for_LDA.to_pickle('news_ready_for_LDA.pkl')